<a href="https://colab.research.google.com/github/rahul897/colab/blob/master/decisiontreeScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
train = [['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

header = ["color", "diameter", "label"]

In [0]:
class Question:
  def __init__(self,col,val):
    self.col = col
    self.val = val
   
  def match(self,row):
    val = row[self.col]
    if isinstance(val, int) or isinstance(val, float):
      return val>=self.val
    else:
      return val==self.val
  
  def __repr__(self):
    condition = "=="
    if isinstance(self.val, int) or isinstance(self.val, float):
        condition = ">="
    return "Is %s %s %s?" % (
        header[self.col], condition, str(self.val))

In [11]:
Question(1, 3)

Is diameter >= 3?

In [0]:
def partition(rows,ques):
  
  tr = list(filter(lambda x:ques.match(x),rows))
  fr = list(filter(lambda x:not ques.match(x),rows))
  
  return tr,fr

In [26]:
true_rows, false_rows = partition(train, Question(0, 'Red'))

true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [0]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [0]:
def gini(rows):
  counts = class_counts(rows)
  impurity = 1
  for lbl in counts:
    prob = counts[lbl]/float(len(rows))
    impurity -= prob**2
  return impurity

In [0]:
def info_gain(left,right,root):
  p = float(len(left))/len(left)+len(right)
  return root-p*gini(left)-(1-p)*gini(right)

In [0]:
def find_best_split(rows):
  best_gain =0
  best_question = None
  root = gini(rows)
  n_features = len(rows[0])-1
  
  for col in range(n_features):
    values = set([row[col] for row in rows])
    for val in values:
      q = Question(col,val)
      
      tr,fr = partition(rows,q)
      
      if len(tr)==0 or len(fr)==0:
        continue
       
      gain = info_gain(tr,fr,root)
      
      if gain>=best_gain:
        best_gain = gain
        best_question = q
  
  return best_gain,best_question

In [0]:
class Decision_Node:

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        

class Leaf:

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [0]:
def build_tree(rows):
  gain,question = find_best_split(rows)
  
  if gain==0:
    return Leaf(rows)
  
  tr,fr = partition(rows,question)
  left = build_tree(tr)
  right = build_tree(fr)
  return Decision_Node(question,left,right)
  
  
def print_tree(node, spacing=""):

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")
  

In [37]:
my_tree = build_tree(train)
  
  
print_tree(my_tree)

Is color == Green?
--> True:
  Predict {'Apple': 1}
--> False:
  Is color == Red?
  --> True:
    Predict {'Grape': 2}
  --> False:
    Predict {'Apple': 1, 'Lemon': 1}


In [0]:
def classify(row, node):

    if isinstance(node, Leaf):
        return node.predictions

    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [40]:
classify(train[0], my_tree)

{'Apple': 1}

In [0]:
def print_leaf(counts):

    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [43]:
print_leaf(classify(train[1], my_tree))

{'Apple': '50%', 'Lemon': '50%'}

In [47]:
test = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

for td in test:
  print('Actual '+td[-1]+" Predicted :"+str(print_leaf(classify(td, my_tree))))

Actual Apple Predicted :{'Apple': '100%'}
Actual Apple Predicted :{'Apple': '50%', 'Lemon': '50%'}
Actual Grape Predicted :{'Grape': '100%'}
Actual Grape Predicted :{'Grape': '100%'}
Actual Lemon Predicted :{'Apple': '50%', 'Lemon': '50%'}
